In [1]:
from random import seed, randint, random
from numpy import array, argmax
from math import ceil, log10, sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, RepeatVector, Input   

In [2]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = [], []
    for i in range(n_examples):
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

In [3]:
X, y = random_sum_pairs(100, 4, 20)
print(X)

[[10, 18, 11, 8], [7, 11, 17, 3], [1, 4, 3, 7], [9, 3, 14, 19], [17, 9, 4, 9], [12, 19, 7, 20], [19, 4, 2, 11], [12, 1, 13, 4], [10, 17, 18, 6], [6, 5, 13, 14], [19, 16, 7, 17], [5, 1, 9, 19], [10, 7, 15, 10], [2, 17, 20, 19], [19, 11, 1, 7], [6, 13, 17, 2], [18, 1, 18, 16], [5, 8, 19, 2], [15, 7, 15, 19], [6, 8, 20, 12], [7, 15, 19, 11], [15, 19, 7, 6], [15, 8, 15, 4], [9, 10, 3, 5], [11, 2, 6, 8], [6, 11, 1, 19], [2, 10, 7, 13], [16, 3, 14, 6], [20, 10, 9, 10], [12, 11, 2, 6], [4, 10, 16, 9], [9, 9, 20, 19], [2, 20, 16, 9], [6, 10, 14, 7], [12, 15, 14, 11], [10, 4, 6, 16], [17, 3, 1, 4], [12, 9, 15, 2], [3, 6, 5, 15], [1, 12, 14, 5], [2, 14, 20, 18], [20, 6, 11, 14], [5, 14, 13, 12], [15, 13, 14, 2], [15, 8, 6, 1], [17, 7, 5, 4], [19, 14, 5, 19], [16, 16, 15, 16], [7, 8, 5, 12], [19, 19, 6, 4], [9, 20, 16, 17], [10, 1, 11, 12], [6, 9, 18, 6], [5, 10, 19, 13], [20, 1, 9, 10], [8, 3, 2, 14], [3, 10, 5, 10], [16, 11, 10, 20], [11, 18, 18, 5], [19, 4, 6, 18], [5, 4, 15, 2], [11, 3, 15, 7

In [4]:
print(y)

[47, 38, 15, 45, 39, 58, 36, 30, 51, 38, 59, 34, 42, 58, 38, 38, 53, 34, 56, 46, 52, 47, 42, 27, 27, 37, 32, 39, 49, 31, 39, 57, 47, 37, 52, 36, 25, 38, 29, 32, 54, 51, 44, 44, 30, 33, 57, 63, 32, 48, 62, 34, 39, 47, 40, 27, 28, 57, 52, 47, 26, 36, 59, 53, 37, 36, 51, 59, 61, 46, 55, 39, 45, 37, 49, 67, 41, 40, 52, 50, 45, 44, 59, 18, 40, 37, 34, 15, 16, 30, 32, 37, 47, 35, 62, 38, 34, 34, 36, 34]


In [5]:
ceil(log10(20+1))

2

In [6]:
def pairs_to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1 # Calculate max length for padding
    Xstr = []
    for p in X:
        strp = '+'.join([str(n) for n in p])
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp # Pad with spaces
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr = []
    for p in y:
        strp = str(p)
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [7]:
Xstr, ystr = pairs_to_string(X, y, 4, 20)

In [8]:
Xstr

[' 10+18+11+8',
 '  7+11+17+3',
 '    1+4+3+7',
 '  9+3+14+19',
 '   17+9+4+9',
 ' 12+19+7+20',
 '  19+4+2+11',
 '  12+1+13+4',
 ' 10+17+18+6',
 '  6+5+13+14',
 ' 19+16+7+17',
 '   5+1+9+19',
 ' 10+7+15+10',
 ' 2+17+20+19',
 '  19+11+1+7',
 '  6+13+17+2',
 ' 18+1+18+16',
 '   5+8+19+2',
 ' 15+7+15+19',
 '  6+8+20+12',
 ' 7+15+19+11',
 '  15+19+7+6',
 '  15+8+15+4',
 '   9+10+3+5',
 '   11+2+6+8',
 '  6+11+1+19',
 '  2+10+7+13',
 '  16+3+14+6',
 ' 20+10+9+10',
 '  12+11+2+6',
 '  4+10+16+9',
 '  9+9+20+19',
 '  2+20+16+9',
 '  6+10+14+7',
 '12+15+14+11',
 '  10+4+6+16',
 '   17+3+1+4',
 '  12+9+15+2',
 '   3+6+5+15',
 '  1+12+14+5',
 ' 2+14+20+18',
 ' 20+6+11+14',
 ' 5+14+13+12',
 ' 15+13+14+2',
 '   15+8+6+1',
 '   17+7+5+4',
 ' 19+14+5+19',
 '16+16+15+16',
 '   7+8+5+12',
 '  19+19+6+4',
 ' 9+20+16+17',
 ' 10+1+11+12',
 '   6+9+18+6',
 ' 5+10+19+13',
 '  20+1+9+10',
 '   8+3+2+14',
 '  3+10+5+10',
 '16+11+10+20',
 ' 11+18+18+5',
 '  19+4+6+18',
 '   5+4+15+2',
 '  11+3+15+7',
 ' 11+20

In [9]:
def add_random_spaces(X: str, max_extra_spaces: int = 3) -> str:
    """
    به یک رشته اسپیس‌های تصادفی اضافه می‌کند.
    - max_extra_spaces = حداکثر تعداد space که ممکنه یکجا اضافه بشه
    """
    new_chars = []
    for ch in X:
        new_chars.append(ch)
        # با احتمال ۳۰٪ بعد از هر کاراکتر چند تا space اضافه کن
        if random() < 0.2:
            new_chars.append(" " * randint(1, max_extra_spaces))

    # با احتمال ۵۰٪ اول رشته چندتا space اضافه کن
    if random() < 0.1:
        new_chars.insert(0, " " * randint(1, max_extra_spaces))

    # با احتمال ۵۰٪ آخر رشته هم چندتا space اضافه کن
    if random() < 0.1:
        new_chars.append(" " * randint(1, max_extra_spaces))

    return "".join(new_chars)

In [10]:
Xstr_noisy = [add_random_spaces(x) for x in Xstr]

In [11]:
def integer_encode(X, y, alphabets):
    char_to_int = dict((c, i) for i, c in enumerate(alphabets))
    print(char_to_int)
    Xenc = []
    for p in X:
        integer_encoded = [char_to_int[char] for char in p]
        Xenc.append(integer_encoded)
    yenc = []
    for p in y:
        integer_encoded = [char_to_int[char] for char in p]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [12]:
Xenc, yenc = integer_encode(Xstr_noisy, ystr, "0123456789+ ")

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '+': 10, ' ': 11}


In [13]:
max_length_Xenc = max(len(x) for x in Xenc)
max_length_Xenc

24

In [14]:
def pad_sequences(encoded_list, max_length, padding_value = 11):
    """
    encoded_list: لیست دنباله‌های integer_encoded
    max_length: طول ثابت نهایی
    padding_value: عددی که برای padding استفاده می‌کنیم (مثلا index اسپیس)
    """
    padded_list = []
    for seq in encoded_list:
        if len(seq) < max_length:
            # کوتاه‌تر: اضافه کردن padding در انتها
            seq = seq + [padding_value] * (max_length - len(seq))
        elif len(seq) > max_length:
            # بلندتر: truncate کردن
            seq = seq[:max_length]
        padded_list.append(seq)
    return padded_list


In [15]:
Xenc_pad = pad_sequences(Xenc, max_length_Xenc)
Xenc_pad

[[11,
  1,
  0,
  11,
  11,
  10,
  1,
  8,
  10,
  1,
  1,
  10,
  8,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [11,
  11,
  11,
  11,
  11,
  11,
  7,
  10,
  11,
  11,
  11,
  1,
  1,
  10,
  1,
  11,
  7,
  10,
  3,
  11,
  11,
  11,
  11,
  11],
 [11,
  11,
  11,
  11,
  1,
  10,
  4,
  11,
  11,
  10,
  3,
  10,
  7,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [11,
  11,
  11,
  11,
  9,
  10,
  3,
  10,
  1,
  4,
  10,
  11,
  1,
  11,
  9,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [11,
  11,
  11,
  1,
  11,
  11,
  11,
  7,
  10,
  11,
  9,
  10,
  4,
  10,
  9,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [11,
  1,
  2,
  10,
  1,
  9,
  10,
  7,
  10,
  2,
  0,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [11,
  11,
  1,
  9,
  10,
  4,
  10,
  2,
  10,
  1,
  1,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11],
 [11,
  11,
  1,
  11,
  1

In [16]:
yenc

[[4, 7],
 [3, 8],
 [1, 5],
 [4, 5],
 [3, 9],
 [5, 8],
 [3, 6],
 [3, 0],
 [5, 1],
 [3, 8],
 [5, 9],
 [3, 4],
 [4, 2],
 [5, 8],
 [3, 8],
 [3, 8],
 [5, 3],
 [3, 4],
 [5, 6],
 [4, 6],
 [5, 2],
 [4, 7],
 [4, 2],
 [2, 7],
 [2, 7],
 [3, 7],
 [3, 2],
 [3, 9],
 [4, 9],
 [3, 1],
 [3, 9],
 [5, 7],
 [4, 7],
 [3, 7],
 [5, 2],
 [3, 6],
 [2, 5],
 [3, 8],
 [2, 9],
 [3, 2],
 [5, 4],
 [5, 1],
 [4, 4],
 [4, 4],
 [3, 0],
 [3, 3],
 [5, 7],
 [6, 3],
 [3, 2],
 [4, 8],
 [6, 2],
 [3, 4],
 [3, 9],
 [4, 7],
 [4, 0],
 [2, 7],
 [2, 8],
 [5, 7],
 [5, 2],
 [4, 7],
 [2, 6],
 [3, 6],
 [5, 9],
 [5, 3],
 [3, 7],
 [3, 6],
 [5, 1],
 [5, 9],
 [6, 1],
 [4, 6],
 [5, 5],
 [3, 9],
 [4, 5],
 [3, 7],
 [4, 9],
 [6, 7],
 [4, 1],
 [4, 0],
 [5, 2],
 [5, 0],
 [4, 5],
 [4, 4],
 [5, 9],
 [1, 8],
 [4, 0],
 [3, 7],
 [3, 4],
 [1, 5],
 [1, 6],
 [3, 0],
 [3, 2],
 [3, 7],
 [4, 7],
 [3, 5],
 [6, 2],
 [3, 8],
 [3, 4],
 [3, 4],
 [3, 6],
 [3, 4]]

In [17]:
max_length_Xenc_pad = max(len(x) for x in Xenc_pad)
max_length_Xenc_pad

24

In [18]:
def one_hot_encode(X, y, max_int):
    Xenc_pad = []
    for p in X:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc_pad.append(pattern)
    yenc_pad = []
    for p in y:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc_pad.append(pattern)
    return Xenc_pad, yenc_pad

In [19]:
def generate_data(n_samples, n_numbers, largest, alphabets, add_noise=False):
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    X, y = pairs_to_string(X, y, n_numbers, largest)

    if add_noise:
        X = [add_random_spaces(x, max_extra_spaces=2) for x in X]  # اسپیس محدود
    # بدون نویز: X = X

    X, y = integer_encode(X, y, alphabets)
    max_length_Xenc = max(len(x) for x in X)  # طول واقعی پس از اعمال اسپیس و integer_encode
    X = pad_sequences(X, max_length_Xenc, padding_value=11)
    X, y = one_hot_encode(X, y, len(alphabets))
    X, y = array(X), array(y)

    return X, y, max_length_Xenc  # max_length_Xenc برمیگرده برای Input shape


In [20]:
def integer_decode(seq, alphabets):
    int_to_char = dict((i, c) for i, c in enumerate(alphabets))
    strings = []
    for p in seq:
        string = int_to_char[argmax(p)]
        strings.append(string)
    return ''.join(strings)

In [21]:
seed(1)
n_samples = 1000
n_numbers = 3
largest = 20
alphabets = '0123456789+ '
n_chars = len(alphabets)
# n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
# n_in_seq_length = max_length_Xenc_pad
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))

In [22]:
n_chars

12

In [23]:
n_out_seq_length

2

In [24]:
X, y, n_in_seq_length = generate_data(n_samples, n_numbers, largest, alphabets, add_noise=False)


{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '+': 10, ' ': 11}


In [25]:
X.shape

(1000, 8, 12)

In [26]:
X[1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [27]:
integer_decode(X[1], '0123456789+ ')

'  9+4+16'

In [28]:

model = Sequential()
model.add(Input(shape=(n_in_seq_length, n_chars)))
model.add(LSTM(128))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))


In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        72,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 2, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 2, 12)          │           780 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,380 (478.05 KB)

 Trainable params: 122,380 (478.05 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
all_histories = []

for i in range(100):
    X, y, n_in_seq_length = generate_data(n_samples, n_numbers, largest, alphabets, add_noise=False)
    print(f"Training on dataset #{i+1}")
    history = model.fit(X, y, epochs=5, batch_size=50, verbose=1)
    all_histories.append(history)


{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '+': 10, ' ': 11}
Training on dataset #1
Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2235 - loss: 2.4214
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2420 - loss: 2.1260
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2440 - loss: 1.9782
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2435 - loss: 1.9162
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2480 - loss: 1.8796
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '+': 10, ' ': 11}
Training on dataset #2
Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3115 - loss: 1.7952
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3230 - loss: 1.7395
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3590 - loss: 1.6797
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3750 - loss: 1.6380
Epoch 5/5

In [42]:
X, y, n_in_seq_length = generate_data(n_samples, n_numbers, largest, alphabets, add_noise=False)
result = model.predict(X, batch_size=50)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '+': 10, ' ': 11}
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [43]:
result[0]

array([[6.75041490e-12, 9.70597158e-10, 2.98540437e-09, 7.87920158e-07,
        9.99844790e-01, 1.53189743e-04, 1.14297507e-06, 9.35133221e-11,
        1.89098642e-13, 6.97846669e-10, 6.48985423e-12, 2.08656226e-13],
       [3.99121258e-04, 1.24604012e-05, 7.34580635e-06, 5.04753416e-06,
        4.97908468e-06, 3.69516874e-06, 1.05534401e-03, 5.42994440e-02,
        8.92746747e-01, 5.14657274e-02, 1.10757796e-07, 3.63818398e-09]],
      dtype=float32)

In [44]:
integer_decode(result[0], alphabets)

'48'

In [45]:
expected = [integer_decode(y1, alphabets) for y1 in y]
predicted = [integer_decode(y1, alphabets) for y1 in result]

In [46]:
list(zip(expected, predicted))

[('48', '48'),
 ('34', '34'),
 ('22', '22'),
 ('31', '31'),
 ('36', '36'),
 ('34', '34'),
 ('13', '13'),
 ('33', '33'),
 ('17', '17'),
 ('46', '46'),
 ('19', '19'),
 ('23', '23'),
 ('46', '46'),
 ('40', '40'),
 (' 9', ' 9'),
 ('45', '45'),
 ('30', '30'),
 ('16', '16'),
 ('27', '27'),
 ('22', '22'),
 ('42', '42'),
 ('19', '19'),
 ('41', '41'),
 ('36', '36'),
 ('27', '27'),
 ('22', '22'),
 ('21', '21'),
 ('45', '45'),
 ('32', '32'),
 ('34', '34'),
 ('28', '28'),
 ('33', '33'),
 ('24', '24'),
 ('41', '41'),
 ('35', '35'),
 ('27', '27'),
 ('40', '40'),
 ('38', '38'),
 ('37', '37'),
 ('36', '36'),
 ('26', '26'),
 ('32', '32'),
 ('18', '18'),
 ('39', '39'),
 ('43', '43'),
 ('24', '24'),
 ('34', '34'),
 ('18', '18'),
 ('37', '37'),
 ('29', '29'),
 ('39', '39'),
 ('34', '34'),
 ('29', '29'),
 ('40', '40'),
 ('36', '36'),
 ('45', '45'),
 ('40', '40'),
 ('34', '34'),
 ('17', '17'),
 ('24', '24'),
 ('17', '17'),
 ('20', '20'),
 ('28', '28'),
 ('40', '40'),
 ('24', '24'),
 ('15', '15'),
 ('15', '1

In [61]:
def predict_sum(input_str, model, alphabets, max_length_Xenc):
    from numpy import array
    from keras.activations import softmax
    
    # 1. Encode کاراکترها به اعداد
    char_to_int = {c:i for i,c in enumerate(alphabets)}
    encoded_input = [char_to_int[c] for c in input_str]
    
    # 2. Pad کردن توالی تا طول max_length_Xenc
    if len(encoded_input) < max_length_Xenc:
        encoded_input += [char_to_int[' ']]*(max_length_Xenc - len(encoded_input))
    else:
        encoded_input = encoded_input[:max_length_Xenc]
    
    # 3. One-hot encode
    n_chars = len(alphabets)
    one_hot_input = []
    for idx in encoded_input:
        vec = [0]*n_chars
        vec[idx] = 1
        one_hot_input.append(vec)
    
    # 4. تبدیل به numpy array و reshape برای مدل
    one_hot_input = array([one_hot_input])  # شکل: (1, max_length_Xenc, n_chars)
    
    # 5. پیش‌بینی
    res = model.predict(one_hot_input)
    
    # 6. Decode خروجی
    int_to_char = {i:c for i,c in enumerate(alphabets)}
    predicted_str = ''
    for timestep in res[0]:
        idx = timestep.argmax()  # index با بیشترین احتمال
        predicted_str += int_to_char[idx]
    
    # 7. محاسبه جمع واقعی از رشته decode شده
    # فقط اعداد رو جدا می‌کنیم و جمع می‌کنیم
    numbers = [int(n) for n in predicted_str.split('+') if n.strip().isdigit()]
    predicted_sum = sum(numbers)
    
    return predicted_str, predicted_sum


In [62]:
s = "10+12+11+16"
predicted_str, predicted_sum = predict_sum(s, model, alphabets, max_length_Xenc)
print("Predicted string:", predicted_str)
print("Predicted sum:", predicted_sum)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted string: 58
Predicted sum: 58


In [47]:
def decode_and_sum(y, alphabets):
    """
    y: one-hot encoded sequences (numpy array)
    alphabets: لیست کاراکترها، مثل ['1','2',...,'+',' ']
    """
    from numpy import argmax
    
    int_to_char = {i: c for i, c in enumerate(alphabets)}
    sums = []
    
    for seq in y:
        # decode رشته
        decoded_str = ''.join([int_to_char[argmax(t)] for t in seq])
        # حذف اسپیس‌ها و جدا کردن اعداد با '+' و جمع زدن
        numbers = [int(n) for n in decoded_str.split('+') if n.strip() != '']
        sums.append(sum(numbers))
        
    return sums


In [48]:
expected_sums = decode_and_sum(y, alphabets)
predicted_sums = decode_and_sum(result, alphabets)

# حالا می‌تونی درصد درست پیش‌بینی‌ها رو ببینی:
accuracy_real_sum = sum(1 for e, p in zip(expected_sums, predicted_sums) if e == p) / len(expected_sums)
print("Real sum accuracy:", accuracy_real_sum)

# یا خطای میانگین:
import numpy as np
mean_error = np.mean(np.abs(np.array(expected_sums) - np.array(predicted_sums)))
print("Mean absolute error:", mean_error)


Real sum accuracy: 0.996
Mean absolute error: 0.005


In [49]:
def sum_accuracy(y_true, y_pred, alphabets):
    """
    y_true, y_pred: one-hot encoded sequences (numpy arrays)
    alphabets: لیست کاراکترها، مثل ['1','2',...,'+',' ']
    """
    from numpy import argmax
    
    correct = 0
    total = len(y_true)
    
    int_to_char = {i: c for i, c in enumerate(alphabets)}
    
    for true_seq, pred_seq in zip(y_true, y_pred):
        # decode رشته‌ها
        true_str = ''.join([int_to_char[argmax(t)] for t in true_seq])
        pred_str = ''.join([int_to_char[argmax(t)] for t in pred_seq])
        
        try:
            # تبدیل به اعداد و محاسبه جمع
            true_sum = sum(int(n) for n in true_str.split('+') if n.strip() != '')
            pred_sum = sum(int(n) for n in pred_str.split('+') if n.strip() != '')
            
            if true_sum == pred_sum:
                correct += 1
        except:
            # اگر رشته خراب بود یا قابل تبدیل نبود، نادیده بگیر
            pass
    
    return correct / total


In [50]:
accuracy_real_sum = sum_accuracy(expected, predicted, alphabets)
print("Real sum accuracy:", accuracy_real_sum)


Real sum accuracy: 1.0


In [51]:
import numpy as np
np.abs(np.array(expected, dtype='int') - np.array(predicted, dtype='int')).sum() / len(expected)

np.float64(0.005)